# CUDA-Based Apriori with Association Rule Generation

This document explains the structure and logic of a CUDA-based Apriori implementation that also generates association rules after finding frequent itemsets. The description details each step without showing explicit code snippets.

---

## 1. Overview

The program performs two major tasks:

1. **Frequent Itemset Mining (Apriori)**:
   - Reads a dataset of transactions where each transaction is a list of integer item IDs.
   - Repeatedly generates candidates of increasing size \(k\) (from single-item sets to larger sets).
   - Uses a GPU kernel to count how many transactions contain each candidate.
   - Filters out candidates that do not meet a minimum support threshold.
   - Stops when no more frequent itemsets can be found.

2. **Association Rule Generation**:
   - Once all frequent itemsets are collected, the program examines each itemset and enumerates all possible rules \(A \rightarrow B\) where \(A\) and \(B\) partition the itemset.
   - Computes metrics such as **confidence** (and optionally **lift**) to determine if the rule meets a given confidence threshold.
   - Outputs each qualifying rule.

---

## 2. Data Reading and Flattening

### 2.1 Reading the Dataset
- The program opens a file where each line represents a single transaction.
- Each line has space-separated integer IDs corresponding to items in that transaction.
- These lines are converted into a two-dimensional vector in memory, where each inner vector represents one transaction.

### 2.2 Flattening Transactions
- To efficiently transfer transactions to the GPU, the program flattens the two-dimensional structure into a single array:
  - A single array stores **all** item IDs from **all** transactions.
  - A separate array of **offsets** holds the starting position of each transaction in the flattened array.
  - Another array stores the **length** of each transaction.
- This arrangement lets a single GPU kernel process transactions by indexing into the flattened array.

---

## 3. Apriori Structure

### 3.1 Generating 1-Item Candidates
- The program collects all **unique** items from the dataset to form the initial set of 1-item candidates.
- Each unique integer ID becomes a single-item candidate.

### 3.2 GPU-Based Support Counting
- For each level \(k\):
  - A batch of candidate itemsets is flattened into a single array (similar to how transactions were flattened).
  - The program launches a GPU kernel where each thread checks whether a given transaction contains each candidate itemset.
  - Support is incremented atomically when a candidate is found in the transaction.
  - After the kernel finishes, the program has the support count for each candidate.

### 3.3 Filtering by Min Support
- A user-specified minimum support fraction is multiplied by the total number of transactions to get the **absolute** support threshold.
- Any candidate whose support is below this threshold is discarded and not used to generate larger candidates.

### 3.4 Generating Larger Candidates
- From the frequent itemsets of size \((k - 1)\), the program forms new candidates of size \(k\) by a naive join:
  - It merges pairs of frequent itemsets that share their first \((k - 2)\) items.
  - This approach does minimal pruning and can produce many candidates in dense data.
- The process repeats: flatten, GPU-count, filter, and join again for the next iteration.

### 3.5 Termination Condition
- The Apriori process terminates when:
  - No new candidate itemsets can be formed, or
  - The newly formed candidate itemsets produce zero frequent sets after filtering.

---

## 4. Storing and Accessing Support Counts

- Throughout the process, the program needs to keep track of the **support counts** for each frequent itemset.
- An approach is to create a **map** from an itemset representation (converted into a string key) to its integer support count.
- When a new batch of frequent itemsets is found, the program inserts or updates the map with each itemset’s support count.
- After the final iteration, all itemsets that survived (i.e., all frequent sets) have an entry in this map.

---

## 5. Association Rule Generation

### 5.1 Subset Enumeration
- After building the **support map** for all frequent itemsets, the program examines each itemset of size at least 2.
- For an itemset \(I\), it generates all non-empty **proper** subsets \(A\). Each subset \(A\) is formed by backtracking or similar enumeration logic (but never the full set \(I\) itself, since \(B\) would be empty).

### 5.2 Rule Construction and Confidence Calculation
- Once a subset \(A\) is extracted, the program sets \(B = I \setminus A\).
- The **confidence** is computed as \(\text{support}(I) / \text{support}(A)\).
- If the confidence meets or exceeds a user-specified threshold, the rule \(A \rightarrow B\) is deemed valid and displayed.

### 5.3 Optional Lift Metric
- If both \(A\) and \(B\) are frequent, the program can compute **lift** using:
  \[
  \text{lift} = \frac{\text{confidence}}{\text{support}(B) / \text{totalTransactions}}.
  \]
- If \(B\) has a non-zero support, the program outputs the lift value for the rule.

### 5.4 Rule Output
- The program prints each valid rule in a human-readable format, indicating the sets \(A\) and \(B\), and providing confidence (and lift, if calculated).
- A count of the total number of rules generated is shown at the end.

---

## 6. Command-Line Arguments

1. **Input File**  
   A file path containing the dataset.  
2. **Min Support Fraction**  
   A floating-point number between 0 and 1 indicating the fraction of total transactions required for an itemset to be frequent.  
3. **Min Confidence**  
   A floating-point number between 0 and 1. Any rule whose confidence is at least this fraction is output.

For instance, if the program is compiled into a binary named `apriori_cuda`, running with:
./apriori_cuda dataset.dat 0.8 0.7

will:
- Read `dataset.dat`,
- Use 80% of the total transactions as the support threshold,
- Require 70% confidence for association rules.

---

## 7. Key Points and Considerations

1. **Naive Subset Checking**  
   The GPU kernel checks if a transaction contains a candidate itemset by a nested loop. This can be slow if transactions or candidates are large. Optimizations like bitsets or sorting-based intersections can help.

2. **Minimal Pruning**  
   The code demonstrates a naive candidate join without thorough subset pruning. Real Apriori typically checks if all sub-subsets are frequent to reduce the combinatorial explosion.

3. **Memory Usage**  
   Flattening and copying large datasets to the GPU can be memory-intensive. Chunk-based approaches might be needed for very large files.

4. **Association Rule Complexity**  
   Generating all non-empty proper subsets for each frequent itemset can be expensive when itemsets are large. This step can be optimized or constrained by maximum itemset sizes.

5. **Lift vs. Other Metrics**  
   The example code shows how to compute lift. Other metrics like leverage or conviction are straightforward to add if needed.

---

## 8. Summary of Execution Flow

1. **Initialization**: Read dataset, flatten transactions, compute `absoluteMinSupport`.
2. **k=1 Pass**:  
   - Build single-item candidates.  
   - GPU count.  
   - Filter by `absoluteMinSupport`.  
   - Store frequent items in a map.  
3. **Recursive Apriori**:  
   - Generate \((k)\)-item candidates from \((k-1)\)-item frequent sets.  
   - GPU count and filter.  
   - Store newly discovered frequent sets in the map.  
   - Stop if no candidates or no frequent sets remain.  
4. **Rule Generation**:  
   - For each frequent itemset in the map, generate subsets \(A\).  
   - Compute confidence for \(A \rightarrow B\).  
   - Output rules that meet `minConfidence`, optionally including lift.  
5. **Completion**:  
   - Print summary of discovered rules, frequent itemsets, etc.

This approach demonstrates how to integrate GPU-accelerated support counting with the standard Apriori algorithm flow, and how to leverage the resulting frequent itemsets to generate association rules in the final step.

In [41]:
%%writefile apriori_CUDA.cu
/*****************************************************************************
 * Example: CUDA-Based Apriori with k-Itemset Generation (Simplified) + Association Rules
 *
 * 1. Reads transactions from a file (each line = one transaction).
 * 2. Builds 1-item candidates, counts on GPU, filters by minSupport.
 * 3. Iteratively generates larger itemsets (k=2, k=3, ...) via naive join.
 * 4. Stops when no more frequent itemsets remain.
 * 5. Generates Association Rules (A -> B) from final frequent itemsets using
 *    a user-specified minConfidence threshold.
 *
 * NOTE:
 *  - Not optimized for very large data (missing advanced pruning, etc.).
 *  - Storing itemsets in a map with string keys for simplicity (avoid custom hashing).
 *****************************************************************************/

#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <sstream>
#include <algorithm>
#include <unordered_map>
#include <unordered_set>
#include <set>
#include <map>
#include <cmath>        // for std::ceil
#include <cuda.h>
#include <cuda_runtime.h>

// ---------------------------------------------------------------------------
// HELPER MACROS
// ---------------------------------------------------------------------------
#define CUDA_CHECK(call)                                                      \
    do {                                                                      \
        cudaError_t err = call;                                              \
        if (err != cudaSuccess) {                                            \
            std::cerr << "CUDA error in " << __FILE__ << ":" << __LINE__     \
                      << ": " << cudaGetErrorString(err) << std::endl;        \
            exit(EXIT_FAILURE);                                              \
        }                                                                     \
    } while (0)

// ---------------------------------------------------------------------------
// CUDA KERNEL for support counting
// Each thread processes one transaction. For each candidate, check if contained
// in that transaction. If yes, atomicAdd to its count.
// ---------------------------------------------------------------------------
__global__ void supportCountKernel(const int* d_transactions,
                                   const int* d_transactionOffsets,
                                   const int* d_transactionLengths,
                                   int numTransactions,
                                   const int* d_candidates,
                                   const int* d_candidateOffsets,
                                   const int* d_candidateLengths,
                                   int numCandidates,
                                   int* d_counts)
{
    int txIndex = blockIdx.x * blockDim.x + threadIdx.x;
    if (txIndex >= numTransactions) return;

    int start = d_transactionOffsets[txIndex];
    int length = d_transactionLengths[txIndex];

    // For each candidate itemset
    for (int c = 0; c < numCandidates; c++) {
        int cStart  = d_candidateOffsets[c];
        int cLength = d_candidateLengths[c];

        // Check if candidate itemset is contained in this transaction
        bool contained = true;
        for (int i = 0; i < cLength && contained; i++) {
            int candidateItem = d_candidates[cStart + i];

            // naive membership check in the transaction
            bool found = false;
            for (int j = 0; j < length && !found; j++) {
                if (d_transactions[start + j] == candidateItem) {
                    found = true;
                }
            }
            if (!found) {
                contained = false;
            }
        }

        if (contained) {
            atomicAdd(&d_counts[c], 1);
        }
    }
}

// ---------------------------------------------------------------------------
// Read dataset from fileName:
//   Each line => space-delimited item IDs (integers).
// ---------------------------------------------------------------------------
std::vector<std::vector<int>> readDataset(const std::string& fileName) {
    std::vector<std::vector<int>> dataset;
    std::ifstream fin(fileName);
    if (!fin.is_open()) {
        std::cerr << "Cannot open file: " << fileName << std::endl;
        return dataset;
    }

    std::string line;
    while (std::getline(fin, line)) {
        if (line.empty()) continue;
        std::stringstream ss(line);
        int item;
        std::vector<int> transaction;
        while (ss >> item) {
            transaction.push_back(item);
        }
        // Optionally sort each transaction to keep a canonical order
        // std::sort(transaction.begin(), transaction.end());
        dataset.push_back(transaction);
    }
    fin.close();
    return dataset;
}

// ---------------------------------------------------------------------------
// Flatten transactions + offsets + lengths for GPU transfer
// ---------------------------------------------------------------------------
void flattenTransactions(const std::vector<std::vector<int>>& dataset,
                         std::vector<int>& h_transactions,
                         std::vector<int>& h_offsets,
                         std::vector<int>& h_lengths)
{
    h_transactions.clear();
    h_offsets.clear();
    h_lengths.clear();

    int offset = 0;
    for (auto &tx : dataset) {
        h_offsets.push_back(offset);
        h_lengths.push_back((int)tx.size());
        for (int item : tx) {
            h_transactions.push_back(item);
        }
        offset += (int)tx.size();
    }
}

// ---------------------------------------------------------------------------
// Generate 1-item candidates from dataset (one item per set)
// ---------------------------------------------------------------------------
std::vector<std::vector<int>> generate1ItemCandidates(const std::vector<std::vector<int>>& dataset)
{
    std::set<int> uniqueItems;
    for (auto &tx : dataset) {
        for (auto &item : tx) {
            uniqueItems.insert(item);
        }
    }

    std::vector<std::vector<int>> cands;
    for (auto &it : uniqueItems) {
        cands.push_back({it});
    }
    return cands;
}

// ---------------------------------------------------------------------------
// Filter cands by minSupport (keeps only those with counts[i] >= minSupport)
// ---------------------------------------------------------------------------
std::vector<std::vector<int>> filterCandidates(const std::vector<std::vector<int>>& cands,
                                               const std::vector<int>& counts,
                                               int minSupport)
{
    std::vector<std::vector<int>> result;
    for (size_t i = 0; i < cands.size(); i++) {
        if (counts[i] >= minSupport) {
            result.push_back(cands[i]);
        }
    }
    return result;
}

// ---------------------------------------------------------------------------
// Naive generation of (k)-item candidates from (k-1)-item frequent sets
//   Merges freq sets that share the first (k-2) items in common.
//   Does not do subset pruning beyond that.
// ---------------------------------------------------------------------------
std::vector<std::vector<int>> generateKItemCandidates(const std::vector<std::vector<int>>& freqItemsets)
{
    std::vector<std::vector<int>> newCands;
    size_t n = freqItemsets.size();
    if (n == 0) return newCands;

    int kMinus1 = (int)freqItemsets[0].size();  // size of each itemset in freqItemsets

    for (size_t i = 0; i < n; i++) {
        for (size_t j = i + 1; j < n; j++) {
            bool canJoin = true;
            // Compare first (k-2) items
            for (int idx = 0; idx < kMinus1 - 1; idx++) {
                if (freqItemsets[i][idx] != freqItemsets[j][idx]) {
                    canJoin = false;
                    break;
                }
            }
            if (canJoin) {
                // Merge freqItemsets[i] and freqItemsets[j]
                std::vector<int> merged = freqItemsets[i];
                int last_i = freqItemsets[i].back();
                int last_j = freqItemsets[j].back();
                if (last_i < last_j) {
                    merged.push_back(last_j);
                } else if (last_i > last_j) {
                    merged.push_back(last_i);
                } else {
                    continue; // identical last item => skip
                }
                std::sort(merged.begin(), merged.end());
                newCands.push_back(merged);
            }
        }
    }

    // remove duplicates if any
    std::sort(newCands.begin(), newCands.end());
    newCands.erase(std::unique(newCands.begin(), newCands.end()), newCands.end());
    return newCands;
}

// ---------------------------------------------------------------------------
// Flatten candidates for GPU (similar concept as flattening transactions)
//   - h_candidates: all item IDs in one array
//   - h_candidateOffsets[i]: where candidate i starts in h_candidates
//   - h_candidateLengths[i]: length of candidate i
// ---------------------------------------------------------------------------
void flattenCandidates(const std::vector<std::vector<int>>& cands,
                       std::vector<int>& h_candidates,
                       std::vector<int>& h_candidateOffsets,
                       std::vector<int>& h_candidateLengths)
{
    h_candidates.clear();
    h_candidateOffsets.clear();
    h_candidateLengths.clear();

    for (auto &v : cands) {
        h_candidateOffsets.push_back((int)h_candidates.size());
        h_candidateLengths.push_back((int)v.size());
        for (auto &item : v) {
            h_candidates.push_back(item);
        }
    }
}

// ---------------------------------------------------------------------------
// GPU-based counting of candidate support
// Returns an array of counts (size = numCandidates)
// ---------------------------------------------------------------------------
std::vector<int> gpuCountSupport(const std::vector<int>& h_transactions,
                                 const std::vector<int>& h_offsets,
                                 const std::vector<int>& h_lengths,
                                 int numTransactions,
                                 const std::vector<int>& h_candidates,
                                 const std::vector<int>& h_candidateOffsets,
                                 const std::vector<int>& h_candidateLengths,
                                 int numCandidates)
{
    // (1) Copy transactions to GPU
    int totalTxItems = (int)h_transactions.size();

    int *d_transactions, *d_txOffsets, *d_txLengths;
    CUDA_CHECK(cudaMalloc((void**)&d_transactions,  totalTxItems * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_txOffsets,     numTransactions * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_txLengths,     numTransactions * sizeof(int)));

    CUDA_CHECK(cudaMemcpy(d_transactions,  h_transactions.data(),
                          totalTxItems * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_txOffsets, h_offsets.data(),
                          numTransactions * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_txLengths, h_lengths.data(),
                          numTransactions * sizeof(int), cudaMemcpyHostToDevice));

    // (2) Copy candidates to GPU
    int totalCandItems = (int)h_candidates.size();
    int *d_candidates, *d_candOffsets, *d_candLengths;
    CUDA_CHECK(cudaMalloc((void**)&d_candidates,    totalCandItems * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_candOffsets,   numCandidates * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_candLengths,   numCandidates * sizeof(int)));

    CUDA_CHECK(cudaMemcpy(d_candidates, h_candidates.data(),
                          totalCandItems * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_candOffsets, h_candidateOffsets.data(),
                          numCandidates * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_candLengths, h_candidateLengths.data(),
                          numCandidates * sizeof(int), cudaMemcpyHostToDevice));

    // (3) Allocate array for counts
    int *d_counts;
    CUDA_CHECK(cudaMalloc((void**)&d_counts, numCandidates * sizeof(int)));
    CUDA_CHECK(cudaMemset(d_counts, 0, numCandidates * sizeof(int)));

    // (4) Launch kernel
    int threadsPerBlock = 128;
    int blocks = (numTransactions + threadsPerBlock - 1) / threadsPerBlock;
    supportCountKernel<<<blocks, threadsPerBlock>>>(
        d_transactions, d_txOffsets, d_txLengths,
        numTransactions,
        d_candidates, d_candOffsets, d_candLengths,
        numCandidates,
        d_counts
    );
    CUDA_CHECK(cudaDeviceSynchronize());

    // (5) Copy results back
    std::vector<int> counts(numCandidates, 0);
    CUDA_CHECK(cudaMemcpy(counts.data(), d_counts,
                          numCandidates * sizeof(int), cudaMemcpyDeviceToHost));

    // (6) Free device memory
    cudaFree(d_transactions);
    cudaFree(d_txOffsets);
    cudaFree(d_txLengths);
    cudaFree(d_candidates);
    cudaFree(d_candOffsets);
    cudaFree(d_candLengths);
    cudaFree(d_counts);

    return counts;
}

// ---------------------------------------------------------------------------
// A small helper to convert an itemset (vector<int>) -> string key
// e.g. [1,4,7] => "1_4_7"
// We'll store these in a map to record their support counts.
// ---------------------------------------------------------------------------
std::string itemsetToString(const std::vector<int>& itemset) {
    if (itemset.empty()) return "";
    std::stringstream ss;
    ss << itemset[0];
    for (size_t i = 1; i < itemset.size(); i++) {
        ss << "_" << itemset[i];
    }
    return ss.str();
}

// ---------------------------------------------------------------------------
// A helper to convert a string key like "1_4_7" back to a vector<int>
// ---------------------------------------------------------------------------
std::vector<int> stringToItemset(const std::string& key) {
    std::vector<int> result;
    if (key.empty()) return result;
    std::stringstream ss(key);
    std::string part;
    while (std::getline(ss, part, '_')) {
        result.push_back(std::stoi(part));
    }
    return result;
}

// ---------------------------------------------------------------------------
// Recursively generate all non-empty proper subsets of 'itemset'
// e.g. itemset = [1,2,3]
// subsets => [1], [2], [3], [1,2], [1,3], [2,3]
// but NOT [1,2,3], and not the empty set
// ---------------------------------------------------------------------------
void generateNonEmptyProperSubsets(const std::vector<int>& itemset,
                                   size_t start,
                                   std::vector<int>& current,
                                   std::vector<std::vector<int>>& results)
{
    // We do a standard backtracking approach
    for (size_t i = start; i < itemset.size(); i++) {
        current.push_back(itemset[i]);
        // record the current subset
        if (current.size() < itemset.size()) {
            // ensure it's a proper subset (not entire itemset)
            results.push_back(current);
        }
        // recurse
        generateNonEmptyProperSubsets(itemset, i + 1, current, results);
        // backtrack
        current.pop_back();
    }
}

// ---------------------------------------------------------------------------
// Generate association rules from all frequent itemsets in 'supportMap'.
//
// Pseudocode:
//   For each frequent itemset I of size >= 2:
//       generate all non-empty subsets A of I (except I itself)
//       let B = I \ A
//       confidence = support(I) / support(A)
//       if confidence >= minConfidence => rule: A -> B
//       optionally compute lift = confidence / support(B)
//
// We store support as #transactions, so 'confidence' => (#(I)) / #(A).
// If you want 'lift', you do (#(I)/#(A)) / (#(B)/#transactions).
//
// 'minConfidence' is a fraction (0.6 => 60%, etc.)
// ---------------------------------------------------------------------------
void generateAssociationRules(const std::unordered_map<std::string,int>& supportMap,
                              int numTransactions,
                              double minConfidence)
{
    std::cout << "\nGenerating association rules (minConfidence = " << minConfidence << ")...\n";

    int ruleCount = 0;

    // We'll iterate through each itemset in supportMap
    for (auto &pair : supportMap) {
        const std::string& key = pair.first;
        int supportCountOfI = pair.second;
        // skip itemsets of size 1 (can't form a meaningful rule)
        auto itemset = stringToItemset(key);
        if (itemset.size() < 2) {
            continue;
        }

        // generate all non-empty subsets A (excluding entire itemset)
        std::vector<std::vector<int>> subsets;
        std::vector<int> current;
        generateNonEmptyProperSubsets(itemset, 0, current, subsets);

        // for each subset A, B = I \ A
        for (auto &A : subsets) {
            // A in string form
            std::string Akey = itemsetToString(A);
            // if A wasn't frequent for some reason, skip
            if (supportMap.find(Akey) == supportMap.end()) {
                continue;
            }
            int supportCountOfA = supportMap.at(Akey);

            // Build B = I \ A
            // easiest is to do a set difference approach
            std::vector<int> B;
            {
                std::unordered_multiset<int> Iset(itemset.begin(), itemset.end());
                for (auto &a : A) {
                    auto it = Iset.find(a);
                    if (it != Iset.end()) {
                        Iset.erase(it); // remove one occurrence
                    }
                }
                // Now what's left in Iset is B
                B.assign(Iset.begin(), Iset.end());
                std::sort(B.begin(), B.end());
            }

            // confidence = support(I) / support(A)
            double conf = double(supportCountOfI) / double(supportCountOfA);

            if (conf >= minConfidence) {
                // optional: compute lift = conf / (support(B)/N)
                // but only if B in supportMap
                double lift = 0.0;
                if (!B.empty()) {
                    std::string Bkey = itemsetToString(B);
                    if (supportMap.find(Bkey) != supportMap.end()) {
                        int supportCountOfB = supportMap.at(Bkey);
                        double supportB = double(supportCountOfB) / double(numTransactions);
                        if (supportB > 0.0) {
                            lift = conf / supportB;
                        }
                    }
                }

                // Print the rule
                ruleCount++;
                // Convert A, B to a nicer string
                std::cout << "Rule #" << ruleCount << ": {";
                for (size_t i = 0; i < A.size(); i++) {
                    std::cout << A[i] << (i+1 < A.size() ? "," : "");
                }
                std::cout << "} => {";
                for (size_t i = 0; i < B.size(); i++) {
                    std::cout << B[i] << (i+1 < B.size() ? "," : "");
                }
                std::cout << "}  "
                          << "conf=" << conf;
                if (lift > 0.0) {
                    std::cout << ", lift=" << lift;
                }
                std::cout << std::endl;
            }
        }
    }

    std::cout << "Total association rules generated: " << ruleCount << std::endl;
}

// ---------------------------------------------------------------------------
// MAIN
// ---------------------------------------------------------------------------
int main(int argc, char** argv) {
    // ----------------------------------------------------------------------
    // 1) Parse command-line arguments or set defaults
    //    - We'll interpret 'minSupportFrac' as fraction of total transactions
    //    - We'll interpret 'minConfidence' as fraction (0.6 => 60% confidence)
    // ----------------------------------------------------------------------
    std::string fileName    = "/content/chess.dat";
    double minSupportFrac   = 0.95;   // e.g. 50% of transactions
    double minConfidence    = 0.8;   // e.g. 60%

    if (argc >= 2) fileName       = argv[1];
    if (argc >= 3) minSupportFrac = std::stod(argv[2]); // e.g. 0.8
    if (argc >= 4) minConfidence  = std::stod(argv[3]); // e.g. 0.7

    std::cout << "Reading dataset from " << fileName << std::endl;
    auto dataset = readDataset(fileName);
    if (dataset.empty()) {
        std::cerr << "Dataset is empty or could not be read.\n";
        return 1;
    }
    int numTransactions = (int)dataset.size();
    std::cout << "Number of transactions: " << numTransactions << std::endl;

    // Flatten dataset for GPU
    std::vector<int> h_transactions, h_offsets, h_lengths;
    flattenTransactions(dataset, h_transactions, h_offsets, h_lengths);

    // Convert fractional minSupport to absolute
    int absoluteMinSupport = (int)std::ceil(minSupportFrac * numTransactions);
    std::cout << "Minimum support fraction: " << minSupportFrac
              << " => absolute minSupport: " << absoluteMinSupport << "\n";
    std::cout << "Minimum confidence: " << minConfidence << "\n\n";

    // ----------------------------------------------------------------------
    // 2) Generate frequent 1-itemsets
    // ----------------------------------------------------------------------
    std::cout << "Generating 1-item candidates...\n";
    auto oneItemCandidates = generate1ItemCandidates(dataset);
    if (oneItemCandidates.empty()) {
        std::cout << "No 1-item candidates found. Stopping.\n";
        return 0;
    }

    // We'll keep a global map of itemset -> supportCount (for rule generation)
    std::unordered_map<std::string,int> supportMap;

    // Flatten and GPU-count
    {
        std::vector<int> h_candidates, h_candidateOffsets, h_candidateLengths;
        flattenCandidates(oneItemCandidates, h_candidates, h_candidateOffsets, h_candidateLengths);

        int numCands = (int)oneItemCandidates.size();
        auto counts = gpuCountSupport(
            h_transactions, h_offsets, h_lengths, numTransactions,
            h_candidates, h_candidateOffsets, h_candidateLengths, numCands
        );

        // Filter out infrequent
        auto freqSets = filterCandidates(oneItemCandidates, counts, absoluteMinSupport);
        std::cout << "Generated " << oneItemCandidates.size()
                  << " candidates, found " << freqSets.size()
                  << " frequent itemsets at k=1\n\n";

        if (freqSets.empty()) {
            std::cout << "No frequent 1-itemsets. Stopping.\n";
            return 0;
        }

        // Store them in our map (with their support counts)
        for (size_t i = 0; i < oneItemCandidates.size(); i++) {
            if (counts[i] >= absoluteMinSupport) {
                auto &cand = oneItemCandidates[i];
                supportMap[itemsetToString(cand)] = counts[i];
            }
        }

        // We'll keep track of allFrequent to iterate next passes
        std::vector<std::vector<int>> currentFreq = freqSets;

        // ------------------------------------------------------------------
        // 3) Main loop for k=2,3,4,... until no more frequent sets
        // ------------------------------------------------------------------
        int k = 2;
        while (true) {
            // generate next-level (k-item) candidates
            auto nextCands = generateKItemCandidates(currentFreq);
            if (nextCands.empty()) {
                std::cout << "No more candidates at k=" << k << ". Stopping.\n";
                break;
            }

            // flatten and count support on GPU
            std::vector<int> hcand, hcandOffsets, hcandLengths;
            flattenCandidates(nextCands, hcand, hcandOffsets, hcandLengths);

            int numC = (int)nextCands.size();
            auto cCounts = gpuCountSupport(
                h_transactions, h_offsets, h_lengths, numTransactions,
                hcand, hcandOffsets, hcandLengths, numC
            );

            // filter by minSupport
            auto newFreq = filterCandidates(nextCands, cCounts, absoluteMinSupport);
            std::cout << "Generated " << nextCands.size()
                      << " candidates, found " << newFreq.size()
                      << " frequent itemsets at k=" << k << "\n";

            if (newFreq.empty()) {
                std::cout << "No more frequent itemsets at k=" << k << ". Stopping.\n";
                break;
            }

            // store them in support map
            for (size_t i = 0; i < nextCands.size(); i++) {
                if (cCounts[i] >= absoluteMinSupport) {
                    auto &cand = nextCands[i];
                    supportMap[itemsetToString(cand)] = cCounts[i];
                }
            }

            // next iteration
            currentFreq = newFreq;
            k++;
            std::cout << std::endl;
        }
    }

    // ----------------------------------------------------------------------
    // 4) Generate association rules from all frequent itemsets
    // ----------------------------------------------------------------------
    generateAssociationRules(supportMap, numTransactions, minConfidence);

    std::cout << "Done.\n";
    return 0;
}

Overwriting apriori_CUDA.cu


In [42]:
!nvcc -o apriori_CUDA apriori_CUDA.cu -lstdc++

In [43]:
!./apriori_CUDA

Reading dataset from /content/chess.dat
Number of transactions: 3196
Minimum support fraction: 0.95 => absolute minSupport: 3037
Minimum confidence: 0.8

Generating 1-item candidates...
Generated 75 candidates, found 9 frequent itemsets at k=1

Generated 36 candidates, found 24 frequent itemsets at k=2

Generated 37 candidates, found 28 frequent itemsets at k=3

Generated 20 candidates, found 14 frequent itemsets at k=4

Generated 2 candidates, found 2 frequent itemsets at k=5

No more candidates at k=6. Stopping.

Generating association rules (minConfidence = 0.8)...
Rule #1: {29} => {40,52,58,60}  conf=0.974222, lift=1.00052
Rule #2: {29,40} => {52,58,60}  conf=0.98225, lift=1.00072
Rule #3: {29,40,52} => {58,60}  conf=0.985687, lift=1.00072
Rule #4: {29,40,52,58} => {60}  conf=0.986001, lift=1.00072
Rule #5: {29,40,52,60} => {58}  conf=0.999677, lift=0.99999
Rule #6: {29,40,58} => {52,60}  conf=0.982562, lift=1.00072
Rule #7: {29,40,58,60} => {52}  conf=0.996463, lift=0.999904
Rule 

In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [1]:
!nvidia-smi

Sat Dec 21 13:31:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/12.3.1/local_installers/cuda-repo-ubuntu2204-12-3-local_12.3.1-545.23.08-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2204-12-3-local_12.3.1-545.23.08-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu2204-12-3-local/cuda-*-keyring.gpg /usr/share/keyrings/
!apt-get update
!apt-get -y install cuda-toolkit-12-3

--2024-12-21 13:21:05--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2204.pin’

cuda-ubuntu2204.pin 100%[===================>]     190  --.-KB/s    in 0s      

2024-12-21 13:21:05 (8.51 MB/s) - ‘cuda-ubuntu2204.pin’ saved [190/190]

--2024-12-21 13:21:05--  https://developer.download.nvidia.com/compute/cuda/12.3.1/local_installers/cuda-repo-ubuntu2204-12-3-local_12.3.1-545.23.08-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O